In [ ]:
#Install bert package for tensorflow v1
!pip install bert-tensorflow==1.0.1
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

from datetime import datetime
import keras
from keras import layers
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tqdm.notebook import tqdm #adds progress bars to show loop status
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def getEmbeddings(df1, df2, trainData, index, classifier):

    """A funtion to extract vector representations (embeddings) from pooled output of BERT. The function first fetches these representations
    for all rows in the dataset df2(with text split applied to all reports), combines the splits from the same report using index 
    (index from originial reports data is preserved for all split chunks) and then maps the list of vector representations with corresonding 
    industry label from dataset df1(without text split applied to all reports) and report name from trainData(initial dataframe read from csv)
    """

    df_emb = np.apply_along_axis(classifier.GetPrediction, 0,np.array(df2["text"]))   #extracts vector representations

    #combines vector representations belonging to single report
    aux = -1
    len_l = 0
    x = {}
    for l, emb in zip(index, df_emb):
        if l in x.keys():
            x[l]  =np.vstack([x[l], emb])
        else:
            x[l] = [emb]

    #maps list of vector representations with corresponding industry labels and report names
    l_final = []
    label_l_final = []
    name = []
    for k in x.keys():
        l_final.append(x[k])
        label_l_final.append(df1.loc[k]['Industry'])
        name.append(trainData.Name.loc[k])

    df = pd.DataFrame({'Name': name, 'emb': l_final, 'label': label_l_final})

    return df